In [ ]:
%reload_ext autoreload
%autoreload 2

#%matplotlib notebook
%matplotlib inline

import numpy as np
import matplotlib.pylab as plt

import gtsam

# Wall detection: passing wall distance and/or angle estimates into GTSAM

Idea is to pass distance / angle estimates obtained from audio, to the drone

In [ ]:
## TODO(FD) adopt below example to a simple use cases with wall measurements and multimodal 
## probability distributions
import sys
sys.path.append('../../src/audio_gtsam/')
from audio_gtsam.wall_backend import WallBackend
from helpers import WallSimulation, plot_all

times = np.arange(10)

wall_backend = WallBackend()

plane = gtsam.OrientedPlane3(gtsam.Unit3([0, -1.0, 0]), 1)
pose_0 = gtsam.Pose3()
wall_simulation = WallSimulation()
wall_simulation.initialize(plane, pose_0)

for t, time in enumerate(times):
    wall_simulation.move_until_time(time)
    
    pose_t = wall_simulation.measure_pose()
    #pose_t = wall_simulation.expected_pose()
    
    plane_meas, azimuth, elevation, distance = wall_simulation.measure_plane()
    
    #print(f"prior factor with {pose_t.translation()}")
    wall_backend.add_pose(r_world=pose_t.translation(), 
                          yaw=pose_t.rotation().yaw())
    wall_angle = 180 + azimuth * 180 / np.pi 
    print(f"adding plane measurement: {distance*100:.0f}cm {wall_angle:5.0f}deg")
    wall_backend.add_plane(distance, azimuth, elevation)
    
    planes, poses = wall_backend.get_results()
    #plot_all(new_result, top=False, side=False)
    #plt.show()
plot_all(wall_backend.result, top=False, side=False)
plt.show()

# DOA algorithms: passing spatial spectrum into GTSAM

Idea is to pass multiple sidelobes from spectrum as individual measurements to the same sound source.  